### Import

In [ ]:
import pandas as pd
import numpy as np

import os
from tqdm import tqdm
import json
import shutil

# image data
from PIL import Image
from glob import glob
# import cv2

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
!nvidia-smi

### 1. EDA

In [ ]:
os.getcwd()

In [ ]:
# define paths for ART, PRE train datas

ART_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly/ART/'
PRE_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly/PRE/'

print("ART N: ", len(os.listdir(ART_path)))
print("PRE N: ", len(os.listdir(PRE_path)))

In [ ]:
# dcm header check
import pydicom

path__ = '/home/ncp/workspace/202002n035/035.신장암 진단을 위한 의료 영상 데이터/01.데이터/신장암2/1.Training/원천데이터/02_01_0010/02_01_0010_ART/0001.dcm'
header_01 = pydicom.dcmread(path__, stop_before_pixels = True)
header_01

### 2. Create label 

In [ ]:
# create ART_train_label dataframe

LABEL_path = '/home/ncp/workspace/202002n035/035.신장암 진단을 위한 의료 영상 데이터/01.데이터/신장암2/1.Training/라벨링데이터/'
people = os.listdir(LABEL_path) 
ART_train_label = pd.DataFrame()

for ppl in tqdm(people): 
    ppl_ARTlabel_path = LABEL_path + ppl + '/' + '{}_ART.json'.format(ppl)
    with open(ppl_ARTlabel_path) as f: 
        ART_json = json.load(f)
    case_id = ART_json['case_id'][0:10]
    type = ART_json['Clinical Information (Global)']['histologic type']
    data = {'case_id' : [case_id], 'histologic type' : [type]}
    data = pd.DataFrame(data)
    ART_train_label = pd.concat([ART_train_label, data]).reset_index(drop = True)

In [ ]:
# create ART_val_label dataframe

LABEL_path = '/home/ncp/workspace/202002n035/035.신장암 진단을 위한 의료 영상 데이터/01.데이터/신장암2/2.Validation/라벨링데이터/'
people = os.listdir(LABEL_path) 
ART_val_label = pd.DataFrame()

for ppl in tqdm(people): 
    ppl_ARTlabel_path = LABEL_path + ppl + '/' + '{}_ART.json'.format(ppl)
    with open(ppl_ARTlabel_path) as f: 
        ART_json = json.load(f)
    case_id = ART_json['case_id'][0:10]
    type = ART_json['Clinical Information (Global)']['histologic type']
    data = {'case_id' : [case_id], 'histologic type' : [type]}
    data = pd.DataFrame(data)
    ART_val_label = pd.concat([ART_val_label, data]).reset_index(drop = True)

In [ ]:
# create PRE_train_label dataframe

LABEL_path = '/home/ncp/workspace/202002n035/035.신장암 진단을 위한 의료 영상 데이터/01.데이터/신장암2/1.Training/라벨링데이터/'
people = os.listdir(LABEL_path) 
PRE_train_label = pd.DataFrame()

for ppl in tqdm(people): 
    ppl_PRElabel_path = LABEL_path + ppl + '/' + '{}_PRE.json'.format(ppl)
    with open(ppl_PRElabel_path) as f: 
        PRE_json = json.load(f)
    case_id = PRE_json['case_id'][0:10]
    type = PRE_json['Clinical Information (Global)']['histologic type']
    data = {'case_id' : [case_id], 'histologic type' : [type]}
    data = pd.DataFrame(data)
    PRE_train_label = pd.concat([PRE_train_label, data]).reset_index(drop = True)

In [ ]:
# create PRE_val_label dataframe

LABEL_path = '/home/ncp/workspace/202002n035/035.신장암 진단을 위한 의료 영상 데이터/01.데이터/신장암2/2.Validation/라벨링데이터/'
people = os.listdir(LABEL_path) 
PRE_val_label = pd.DataFrame()

for ppl in tqdm(people): 
    ppl_PRElabel_path = LABEL_path + ppl + '/' + '{}_PRE.json'.format(ppl)
    with open(ppl_PRElabel_path) as f: 
        PRE_json = json.load(f)
    case_id = PRE_json['case_id'][0:10]
    type = PRE_json['Clinical Information (Global)']['histologic type']
    data = {'case_id' : [case_id], 'histologic type' : [type]}
    data = pd.DataFrame(data)
    PRE_val_label = pd.concat([PRE_val_label, data]).reset_index(drop = True)

In [ ]:
ART_train_label['histologic type'].unique()

In [ ]:
# create new label in label_df
    # cc, chr, pp -> RCC
    # AML
    # ONCO
# (07.28 수정 반영) RCC // AML + onco 로 binary classification

def new_label(df): 
    df['label'] = df['histologic type']
    df.loc[df['label'].isin(['cc', 'chr', 'pp']) == True, 'label'] = 'RCC'
    df.loc[df['label'] == 'AML', 'label'] = 'AML + onco'
    df.loc[df['label'] == 'onco', 'label'] = 'AML + onco'
    return df

ART_train_label = new_label(ART_train_label)
PRE_train_label = new_label(PRE_train_label)
ART_val_label = new_label(ART_val_label)
PRE_val_label = new_label(PRE_val_label)

In [ ]:
# count

print("ART_train_label count: ")
print(ART_train_label['label'].value_counts())
print("PRE_train_label count: ")
print(PRE_train_label['label'].value_counts())

In [ ]:
# count

print("ART_val_label count: ")
print(ART_val_label['label'].value_counts())
print("PRE_val_label count: ")
print(PRE_val_label['label'].value_counts())

In [ ]:
# kidneyData_windowing_mass_classified 폴더는 만들고 돌려야 함
shutil.rmtree('/home/ncp/workspace/blocks1/kidneyData_windowing_mass_classified/TRAIN/')
shutil.rmtree('/home/ncp/workspace/blocks1/kidneyData_windowing_mass_classified/TEST/')

In [ ]:
# 기존 kidney_data - ART, PRE - Test, Train imageset 을 2개의 label별 폴더로 분류 (TRAIN/TEST 분리까지 포함)

ART_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_mass_total/ART/'
PRE_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_mass_total/PRE/'
GCCT_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_mass_total/GCCT/'

new_ART_train_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_mass_classified/TRAIN/ART/'
new_ART_test_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_mass_classified/TEST/ART/'
new_PRE_train_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_mass_classified/TRAIN/PRE/'
new_PRE_test_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_mass_classified/TEST/PRE/'
new_GCCT_train_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_mass_classified/TRAIN/GCCT/'
new_GCCT_test_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_mass_classified/TEST/GCCT/'

# make dir
def makedir(path): 
    if not os.path.exists(path): 
        os.mkdir(path)
        
dir = '/home/ncp/workspace/blocks1/kidneyData_windowing_mass_classified/'
makedir(os.path.join(dir, "TRAIN"))
makedir(os.path.join(dir, "TEST"))

makedir(os.path.join(dir, "TRAIN", "ART"))
makedir(os.path.join(dir, "TRAIN", "ART", "RCC"))
makedir(os.path.join(dir, "TRAIN", "ART", "AML + onco"))

makedir(os.path.join(dir, "TEST", "ART"))
makedir(os.path.join(dir, "TEST", "ART", "RCC"))
makedir(os.path.join(dir, "TEST", "ART", "AML + onco"))

makedir(os.path.join(dir, "TRAIN", "PRE"))
makedir(os.path.join(dir, "TRAIN", "PRE", "RCC"))
makedir(os.path.join(dir, "TRAIN", "PRE", "AML + onco"))

makedir(os.path.join(dir, "TEST", "PRE"))
makedir(os.path.join(dir, "TEST", "PRE", "RCC"))
makedir(os.path.join(dir, "TEST", "PRE", "AML + onco"))

makedir(os.path.join(dir, "TRAIN", "GCCT"))
makedir(os.path.join(dir, "TRAIN", "GCCT", "RCC"))
makedir(os.path.join(dir, "TRAIN", "GCCT", "AML + onco"))

makedir(os.path.join(dir, "TEST", "GCCT"))
makedir(os.path.join(dir, "TEST", "GCCT", "RCC"))
makedir(os.path.join(dir, "TEST", "GCCT", "AML + onco"))


# create img name list
art_list = os.listdir(ART_path)
pre_list = os.listdir(PRE_path)
gcct_list = os.listdir(GCCT_path)


# ART
for img in tqdm(art_list): 
    case_id = img[0:10]
    if case_id[6:10] <= '0320': # train
        label = ART_train_label.loc[ART_train_label['case_id'] == case_id, 'label'].unique().tolist()
        if label == ['RCC']: 
            shutil.copy(ART_path + img, new_ART_train_path + 'RCC/' + img)
        if label == ['AML + onco']: 
            shutil.copy(ART_path + img, new_ART_train_path + 'AML + onco/' + img)
    else: # test
        label = ART_val_label.loc[ART_val_label['case_id'] == case_id, 'label'].unique().tolist()
        if label == ['RCC']: 
            shutil.copy(ART_path + img, new_ART_test_path + 'RCC/' + img)
        if label == ['AML + onco']: 
            shutil.copy(ART_path + img, new_ART_test_path + 'AML + onco/' + img)

# PRE
for img in tqdm(pre_list): 
    case_id = img[0:10]
    if case_id[6:10] <= '0320': # train
        label = PRE_train_label.loc[PRE_train_label['case_id'] == case_id, 'label'].unique().tolist()
        if label == ['RCC']: 
            shutil.copy(PRE_path + img, new_PRE_train_path + 'RCC/' + img)
        if label == ['AML + onco']: 
            shutil.copy(PRE_path + img, new_PRE_train_path + 'AML + onco/' + img)
    else: # test
        label = PRE_val_label.loc[PRE_val_label['case_id'] == case_id, 'label'].unique().tolist()
        if label == ['RCC']: 
            shutil.copy(PRE_path + img, new_PRE_test_path + 'RCC/' + img)
        if label == ['AML + onco']: 
            shutil.copy(PRE_path + img, new_PRE_test_path + 'AML + onco/' + img)
        
# GCCT
for img in tqdm(gcct_list): 
    case_id = img[0:10]
    if case_id[6:10] <= '0320': # train
        label = PRE_train_label.loc[PRE_train_label['case_id'] == case_id, 'label'].unique().tolist()
        if label == ['RCC']: 
            shutil.copy(GCCT_path + img, new_GCCT_train_path + 'RCC/' + img)
        if label == ['AML + onco']: 
            shutil.copy(GCCT_path + img, new_GCCT_train_path + 'AML + onco/' + img)
    else: # test
        label = PRE_val_label.loc[PRE_val_label['case_id'] == case_id, 'label'].unique().tolist()
        if label == ['RCC']: 
            shutil.copy(GCCT_path + img, new_GCCT_test_path + 'RCC/' + img)
        if label == ['AML + onco']: 
            shutil.copy(GCCT_path + img, new_GCCT_test_path + 'AML + onco/' + img)

In [ ]:
# check len of each labels - Train/Test 분리되었을 시

print("ART Train : ")
print("RCC ", len(os.listdir(new_ART_train_path + 'RCC/')))
print("AML + onco ", len(os.listdir(new_ART_train_path + 'AML + onco/')))
print("----------")
print("ART Test : ")
print("RCC ", len(os.listdir(new_ART_test_path + 'RCC/')))
print("AML + onco ", len(os.listdir(new_ART_test_path + 'AML + onco/')))
print("----------")
print("PRE Train : ")
print("RCC ", len(os.listdir(new_PRE_train_path + 'RCC/')))
print("AML + onco ", len(os.listdir(new_PRE_train_path + 'AML + onco/')))
print("----------")
print("PRE Test : ")
print("RCC ", len(os.listdir(new_PRE_test_path + 'RCC/')))
print("AML + onco ", len(os.listdir(new_PRE_test_path + 'AML + onco/')))
print("----------")
print("GCCT Train : ")
print("RCC ", len(os.listdir(new_GCCT_train_path + 'RCC/')))
print("AML + onco ", len(os.listdir(new_GCCT_train_path + 'AML + onco/')))
print("----------")
print("GCCT Test : ")
print("RCC ", len(os.listdir(new_GCCT_test_path + 'RCC/')))
print("AML + onco ", len(os.listdir(new_GCCT_test_path + 'AML + onco/')))
print("----------")

### 3. Classification

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
# from keras.utils import np_utils
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Activation, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import regularizers, optimizers
import tensorflow as tf
from tensorflow.keras.applications import ResNet50,VGG16,ResNet101, VGG19, DenseNet201,   MobileNetV2 # ,EfficientNetB4
from tensorflow.keras.applications import resnet, vgg16 , vgg19, densenet,  mobilenet_v2 # ,efficientnet
from tensorflow.keras import Model

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import scipy

import PIL
import os

In [ ]:
gpus =tf.config.experimental.list_physical_devices('GPU')
if gpus: 
    # tensorflow가 첫번째 gpu만 사용하도록 제한
    try: 
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e: 
        print(e)

In [ ]:
import sys
print(sys.version)

In [ ]:
print(tf.__version__)
print(keras.__version__)
print(np.__version__)

In [ ]:
tf.test.is_gpu_available()

In [ ]:
#tf.test.is_gpu_available()
#tf.test.is_built_with_cuda()
tf.test.is_built_with_gpu_support()

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
batch_size = 32
img_height = 512
img_width = 512

In [ ]:
# ART_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_classified/TRAIN/ART/'
# ART_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_classified/TEST/ART/'
# PRE_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_classified/TRAIN/PRE/'
# PRE_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_classified/TEST/PRE/'
# # GCCT_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_classified/TRAIN/GCCT/'
# # GCCT_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_classified/TEST/GCCT/'

# image_shape = (512, 512, 3)
# N_CLASSES = 2
# BATCH_SIZE = 32

# train_datagen = ImageDataGenerator(dtype='float32', 
#                                        preprocessing_function=resnet.preprocess_input,
#                                        validation_split = 0.2)

# test_datagen = ImageDataGenerator(dtype='float32', 
#                                   preprocessing_function=resnet.preprocess_input, 
#                                   rescale= 1./255.)

# ART_train_generator = train_datagen.flow_from_directory(ART_train_path,
#                                                    batch_size = BATCH_SIZE,
#                                                    target_size = (512, 512),
#                                                    class_mode = 'categorical', 
#                                                     subset = 'training')
# ART_val_generator = train_datagen.flow_from_directory(ART_train_path,
#                                                    batch_size = BATCH_SIZE,
#                                                    target_size = (512, 512),
#                                                    class_mode = 'categorical', 
#                                                     subset = 'validation')
# ART_test_generator = test_datagen.flow_from_directory(ART_test_path,
#                                                    batch_size = BATCH_SIZE,
#                                                    target_size = (512, 512),
#                                                    class_mode = 'categorical')
                                                    

# PRE_train_generator = train_datagen.flow_from_directory(PRE_train_path,
#                                                    batch_size = BATCH_SIZE,
#                                                    target_size = (512, 512),
#                                                    class_mode = 'categorical', 
#                                                     subset = 'training')
# PRE_val_generator = train_datagen.flow_from_directory(PRE_train_path,
#                                                    batch_size = BATCH_SIZE,
#                                                    target_size = (512, 512),
#                                                    class_mode = 'categorical', 
#                                                     subset = 'validation')
# PRE_test_generator = test_datagen.flow_from_directory(PRE_test_path,
#                                                    batch_size = BATCH_SIZE,
#                                                    target_size = (512, 512),
#                                                    class_mode = 'categorical')

# GCCT_train_generator = train_datagen.flow_from_directory(GCCT_train_path,
#                                                    batch_size = BATCH_SIZE,
#                                                    target_size = (512, 512),
#                                                    class_mode = 'categorical', 
#                                                     subset = 'training')
# GCCT_val_generator = train_datagen.flow_from_directory(GCCT_train_path,
#                                                    batch_size = BATCH_SIZE,
#                                                    target_size = (512, 512),
#                                                    class_mode = 'categorical', 
#                                                     subset = 'validation')
# GCCT_test_generator = test_datagen.flow_from_directory(GCCT_test_path,
#                                                    batch_size = BATCH_SIZE,
#                                                    target_size = (512, 512),
#                                                    class_mode = 'categorical')

In [ ]:
# # 그냥 convolution block 쌓아서 해보자

# model = Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = image_shape))
# model.add(layers.MaxPooling2D((2,2)))
# model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
# model.add(layers.MaxPooling2D((2,2)))
# model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
# model.add(layers.Flatten())
# model.add(layers.Dense(64, activation = 'relu'))
# model.add(layers.Dense(N_CLASSES, activation = 'softmax'))

# model.summary()

In [ ]:
# optimizer = optimizers.Adam(learning_rate= 0.00001, decay= 1e-6)
# model.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics = ['acc'])

In [ ]:
# # ART train
# checkpointer = ModelCheckpoint(filepath='./mass_ART.hdf5',
#                             monitor='val_loss', verbose = 1,
#                             save_best_only=True)
# early_stopping = EarlyStopping(verbose=1, patience=15)

# history_res = model.fit(ART_train_generator,
#                     steps_per_epoch = 20,
#                     epochs = 100,
#                     verbose = 1,
#                     validation_data = ART_val_generator,
#                     callbacks = [checkpointer, early_stopping])

### RESNET50

In [ ]:
# res_model = ResNet50(include_top=False, pooling='avg', weights='imagenet', input_shape = (image_shape))
# for layer in res_model.layers:
#     if 'conv5' not in layer.name:
#         layer.trainable = False

In [ ]:
res_model = ResNet50(include_top=False, pooling='avg', weights=None, input_shape = (image_shape))
for layer in res_model.layers:
    if 'conv5' not in layer.name:
        layer.trainable = False

In [ ]:
model = Sequential()
model.add(res_model)
model.add(Dropout(0.6))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dropout(0.6))
model.add(Dense(N_CLASSES, activation='softmax'))
model.summary()

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
optimizer = optimizers.Adam(learning_rate= 0.00001, decay= 1e-6)


learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                           patience =3, 
                                           verbose =1,
                                            factor = 0.2,
                                            min_lr =0.0000001)
model.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics = ['acc'])

In [ ]:
# ART train
checkpointer = ModelCheckpoint(filepath='./mass_ART-ResNet50.hdf5',
                           monitor='val_loss', verbose = 1,
                           save_best_only=True)
early_stopping = EarlyStopping(verbose=1, patience=15)

history_res = model.fit(ART_train_generator,
                    steps_per_epoch = 20,
                    epochs = 100,
                    verbose = 1,
                    validation_data = ART_val_generator,
                    callbacks = [checkpointer, early_stopping,learning_rate_reduction])                   

In [ ]:
result = model.evaluate(ART_test_generator)

In [ ]:
plt.plot(history_res.history['acc'], label = 'train',)
plt.plot(history_res.history['val_acc'], label = 'val')

plt.legend(loc = 'right')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

In [ ]:
# PRE train

checkpointer = ModelCheckpoint(filepath='./cancer_PRE-ResNet50.hdf5',
                            monitor='val_loss', verbose = 1,
                            save_best_only=True)
early_stopping = EarlyStopping(verbose=1, patience=15)

history_res = model.fit(PRE_train_generator,
                    steps_per_epoch = 20,
                    epochs = 50,
                    verbose = 1,
                    validation_data = PRE_val_generator,
                    callbacks = [checkpointer, early_stopping,learning_rate_reduction])

In [ ]:
result = model.evaluate(PRE_test_generator)

In [ ]:
plt.plot(history_res.history['acc'], label = 'train',)
plt.plot(history_res.history['val_acc'], label = 'val')

plt.legend(loc = 'right')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

## compare

In [ ]:
os.getcwd()

In [ ]:
path_resnet_ART = '/home/ncp/workspace/blocks3/zio_code/cancer_ART-ResNet50.hdf5'
ART_resnet = tf.keras.models.load_model(path_resnet_ART)

In [ ]:
path_resnet_PRE = '/home/ncp/workspace/blocks3/zio_code/cancer_PRE-ResNet50.hdf5'
PRE_resnet = tf.keras.models.load_model(path_resnet_PRE)

### 2. Create label - with configuration image           
run '2. Create label' section first to create label dataframes

In [ ]:
# for GCCT 

GCCT_train_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_mass/TRAIN/GCCT/'
GCCT_test_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_mass/TEST/GCCT/'

new_GCCT_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass/TRAIN/GCCT/'
new_GCCT_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass/TEST/GCCT/'

def makedir(path): 
    if not os.path.exists(path): 
        os.mkdir(path)
        
dir = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass/'
makedir(dir)
makedir(os.path.join(dir, "TRAIN"))
makedir(os.path.join(dir, "TEST"))

makedir(os.path.join(dir, "TRAIN", "GCCT"))
makedir(os.path.join(dir, "TRAIN", "GCCT", "RCC"))
makedir(os.path.join(dir, "TRAIN", "GCCT", "AML + onco"))

makedir(os.path.join(dir, "TEST", "GCCT"))
makedir(os.path.join(dir, "TEST", "GCCT", "RCC"))
makedir(os.path.join(dir, "TEST", "GCCT", "AML + onco"))

# create img name list
GCCT_train_list = os.listdir(GCCT_train_path)
GCCT_test_list = os.listdir(GCCT_test_path)

# GCCT train / we can use same label for GCCT and PRE 
for img in tqdm(GCCT_train_list): 
    case_id = img[0:10]
    label = PRE_train_label.loc[PRE_train_label['case_id'] == case_id, 'label'].unique().tolist()
    if label == ['RCC']: 
        shutil.copy(GCCT_train_path + img, new_GCCT_train_path + 'RCC/' + img)
    if label == ['AML + onco']: 
        shutil.copy(GCCT_train_path + img, new_GCCT_train_path + 'AML + onco/' + img)
        
        
# GCCT test
for img in tqdm(GCCT_test_list): 
    case_id = img[0:10]
    label = PRE_val_label.loc[PRE_val_label['case_id'] == case_id, 'label'].unique().tolist()
    if label == ['RCC']: 
        shutil.copy(GCCT_test_path + img, new_GCCT_test_path + 'RCC/' + img)
    if label == ['AML + onco']: 
        shutil.copy(GCCT_test_path + img, new_GCCT_test_path + 'AML + onco/' + img)

In [ ]:
# 기존 kidney_data - ART, PRE - Test, Train imageset 을 3개의 label별 폴더로 분류

ART_train_path = '/home/ncp/workspace/blocks1/kidneyData_windowing/Train_data/ART/'
PRE_train_path = '/home/ncp/workspace/blocks1/kidneyData_windowing/Train_data/PRE/'
ART_test_path = '/home/ncp/workspace/blocks1/kidneyData_windowing/Test_data/ART/'
PRE_test_path = '/home/ncp/workspace/blocks1/kidneyData_windowing/Test_data/PRE/'

new_ART_train_path = '/home/ncp/workspace/blocks3/zio_code/kidneyData_windowing/TRAIN/ART/'
new_ART_test_path = '/home/ncp/workspace/blocks3/zio_code/kidneyData_windowing/TEST/ART/'
new_PRE_train_path = '/home/ncp/workspace/blocks3/zio_code/kidneyData_windowing/TRAIN/PRE/'
new_PRE_test_path = '/home/ncp/workspace/blocks3/zio_code/kidneyData_windowing/TEST/PRE/'

# make dir
def makedir(path): 
    if not os.path.exists(path): 
        os.mkdir(path)
        
dir = '/home/ncp/workspace/blocks3/zio_code/kidneyData_windowing/'
makedir(os.path.join(dir, "TRAIN"))
makedir(os.path.join(dir, "TEST"))

makedir(os.path.join(dir, "TRAIN", "ART"))
makedir(os.path.join(dir, "TRAIN", "ART", "RCC"))
makedir(os.path.join(dir, "TRAIN", "ART", "AML + onco"))

makedir(os.path.join(dir, "TEST", "ART"))
makedir(os.path.join(dir, "TEST", "ART", "RCC"))
makedir(os.path.join(dir, "TEST", "ART", "AML + onco"))

makedir(os.path.join(dir, "TRAIN", "PRE"))
makedir(os.path.join(dir, "TRAIN", "PRE", "RCC"))
makedir(os.path.join(dir, "TRAIN", "PRE", "AML + onco"))

makedir(os.path.join(dir, "TEST", "PRE"))
makedir(os.path.join(dir, "TEST", "PRE", "RCC"))
makedir(os.path.join(dir, "TEST", "PRE", "AML + onco"))


# create img name list
art_train_list = os.listdir(ART_train_path)
pre_train_list = os.listdir(PRE_train_path)
art_test_list = os.listdir(ART_test_path)
pre_test_list = os.listdir(PRE_test_path)


# ART train
for img in tqdm(art_train_list): 
    case_id = img[0:10]
    label = ART_train_label.loc[ART_train_label['case_id'] == case_id, 'label'].unique().tolist()
    if label == ['RCC']: 
        shutil.copy(ART_train_path + img, new_ART_train_path + 'RCC/' + img)
    if label == ['AML + onco']: 
        shutil.copy(ART_train_path + img, new_ART_train_path + 'AML + onco/' + img)

# PRE train
for img in tqdm(pre_train_list): 
    case_id = img[0:10]
    label = PRE_train_label.loc[PRE_train_label['case_id'] == case_id, 'label'].unique().tolist()
    if label == ['RCC']: 
        shutil.copy(PRE_train_path + img, new_PRE_train_path + 'RCC/' + img)
    if label == ['AML + onco']: 
        shutil.copy(PRE_train_path + img, new_PRE_train_path + 'AML + onco/' + img)
        
# ART test
for img in tqdm(art_test_list): 
    case_id = img[0:10]
    label = ART_val_label.loc[ART_val_label['case_id'] == case_id, 'label'].unique().tolist()
    if label == ['RCC']: 
        shutil.copy(ART_test_path + img, new_ART_test_path + 'RCC/' + img)
    if label == ['AML + onco']: 
        shutil.copy(ART_test_path + img, new_ART_test_path + 'AML + onco/' + img)
        
# PRE test
for img in tqdm(pre_test_list): 
    case_id = img[0:10]
    label = PRE_val_label.loc[PRE_val_label['case_id'] == case_id, 'label'].unique().tolist()
    if label == ['RCC']: 
        shutil.copy(PRE_test_path + img, new_PRE_test_path + 'RCC/' + img)
    if label == ['AML + onco']: 
        shutil.copy(PRE_test_path + img, new_PRE_test_path + 'AML + onco/' + img)

In [ ]:
# check len of each labels

print("ART Train : ")
print("RCC ", len(os.listdir(new_ART_train_path + 'RCC/')))
print("AML + onco ", len(os.listdir(new_ART_train_path + 'AML + onco/')))
print("----------")
print("ART Test : ")
print("RCC ", len(os.listdir(new_ART_test_path + 'RCC/')))
print("AML + onco ", len(os.listdir(new_ART_test_path + 'AML + onco/')))
print("----------")
print("PRE Train : ")
print("RCC ", len(os.listdir(new_PRE_train_path + 'RCC/')))
print("AML + onco ", len(os.listdir(new_PRE_train_path + 'AML + onco/')))
print("----------")
print("PRE Test : ")
print("RCC ", len(os.listdir(new_PRE_test_path + 'RCC/')))
print("AML + onco ", len(os.listdir(new_PRE_test_path + 'AML + onco/')))
print("----------")

### 3. Classification - with configuration image

In [ ]:
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential
# from keras.utils import np_utils
from tensorflow.keras.layers import Dense,Activation, Flatten, Dropout, BatchNormalization, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers, optimizers
import tensorflow as tf
from tensorflow.keras.applications import ResNet50,VGG16,ResNet101, VGG19, DenseNet201,   MobileNetV2 # ,EfficientNetB4
from tensorflow.keras.applications import resnet, vgg16 , vgg19, densenet,  mobilenet_v2 # ,efficientnet
from tensorflow.keras import Model

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import scipy

import PIL
import os

In [ ]:
gpus =tf.config.experimental.list_physical_devices('GPU')
if gpus: 
    # tensorflow가 첫번째 gpu만 사용하도록 제한
    try: 
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e: 
        print(e)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
print(gpus[0])

In [ ]:
#tf.test.is_gpu_available()
#tf.test.is_built_with_cuda()
tf.test.is_built_with_gpu_support()

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
batch_size = 32
img_height = 512
img_width = 512

In [ ]:
ART_train_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_classified/TRAIN/ART/'
ART_test_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_classified/TEST/ART/'
PRE_train_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_classified/TRAIN/PRE/'
PRE_test_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_classified/TEST/PRE/'

image_shape = (512, 512, 3)
N_CLASSES = 2
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(dtype='float32', 
                                       rescale= 1./255.,
                                       validation_split = 0.2)

test_datagen = ImageDataGenerator(dtype='float32', 
                                       rescale= 1./255.)

ART_train_generator = train_datagen.flow_from_directory(ART_train_path,
                                                   batch_size = BATCH_SIZE,
                                                   target_size = (512, 512),
                                                   class_mode = 'categorical', 
                                                    subset = 'training')
ART_val_generator = train_datagen.flow_from_directory(ART_train_path,
                                                   batch_size = BATCH_SIZE,
                                                   target_size = (512, 512),
                                                   class_mode = 'categorical', 
                                                    subset = 'validation')
ART_test_generator = test_datagen.flow_from_directory(ART_test_path,
                                                   batch_size = BATCH_SIZE,
                                                   target_size = (512, 512),
                                                   class_mode = 'categorical')
                                                    

PRE_train_generator = train_datagen.flow_from_directory(PRE_train_path,
                                                   batch_size = BATCH_SIZE,
                                                   target_size = (512, 512),
                                                   class_mode = 'categorical', 
                                                    subset = 'training')
PRE_val_generator = train_datagen.flow_from_directory(PRE_train_path,
                                                   batch_size = BATCH_SIZE,
                                                   target_size = (512, 512),
                                                   class_mode = 'categorical', 
                                                    subset = 'validation')
PRE_test_generator = test_datagen.flow_from_directory(PRE_test_path,
                                                   batch_size = BATCH_SIZE,
                                                   target_size = (512, 512),
                                                   class_mode = 'categorical')

### RESNET50

In [ ]:
image_shape = (512, 512, 3)
res_model = ResNet50(include_top=False, pooling='avg', weights='imagenet', input_shape = (image_shape))
for layer in res_model.layers:
    if 'conv5' not in layer.name:
        layer.trainable = False
# Check if all layers except conv5 layers are not trainable
#for i, layer in enumerate(res_model.layers):
#    print(i, layer.name, "-", layer.trainable)

In [ ]:
N_CLASSES = 2
model = Sequential()
model.add(res_model)
model.add(Dropout(0.6))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dropout(0.6))
model.add(Dense(N_CLASSES, activation='softmax'))
model.summary()

In [ ]:
optimizer = optimizers.Adam(learning_rate= 0.00001, decay= 1e-6)
model.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics = ['acc'])

In [ ]:
# ART train
checkpointer = ModelCheckpoint(filepath='./cancer_configuration_ART-ResNet50.hdf5',
                            monitor='val_loss', verbose = 1,
                            save_best_only=True)
early_stopping = EarlyStopping(verbose=1, patience=15)

history_res = model.fit(ART_train_generator,
                    steps_per_epoch = 20,
                    epochs = 100,
                    verbose = 1,
                    validation_data = ART_val_generator,
                    callbacks = [checkpointer, early_stopping])

In [ ]:
tf.isgpu_available()

In [ ]:
result = model.evaluate(ART_test_generator)

In [ ]:
plt.plot(history_res.history['acc'], label = 'train',)
plt.plot(history_res.history['val_acc'], label = 'val')

plt.legend(loc = 'right')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

In [ ]:
history_res.history['acc']

In [ ]:
history_res.history['val_loss']

In [ ]:
model_test = tf.keras.models.load_model('./cancer_configuration_ART-ResNet50.hdf5')
model_test.evaluate(ART_test_generator)

In [ ]:
# PRE train
checkpointer = ModelCheckpoint(filepath='./cancer_configuration_PRE-ResNet50.hdf5',
                            monitor='val_loss', verbose = 1,
                            save_best_only=True)
early_stopping = EarlyStopping(verbose=1, patience=15)

history_res = model.fit(PRE_train_generator,
                    steps_per_epoch = 20,
                    epochs = 100,
                    verbose = 1,
                    validation_data = PRE_val_generator,
                    callbacks = [checkpointer, early_stopping])

In [ ]:
result = model.evaluate(PRE_test_generator)

In [ ]:
plt.plot(history_res.history['acc'], label = 'train',)
plt.plot(history_res.history['val_acc'], label = 'val')

plt.legend(loc = 'right')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

### RESNET101

In [ ]:
image_shape = (512, 512, 3)
res101_model = ResNet101(include_top=False, pooling='avg', weights='imagenet', input_shape = (image_shape))
for layer in res101_model.layers:
    layer.trainable = True

In [ ]:
N_CLASSES = 2

model = Sequential()
model.add(res101_model)
model.add(Dropout(0.6))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(300))
model.add(BatchNormalization())
model.add(Dense(100))
model.add(BatchNormalization())
model.add(Dropout(0.6))
model.add(Dense(N_CLASSES, activation='softmax'))
model.summary()

In [ ]:
import scipy

In [ ]:
# ART Train
optimizer = optimizers.Adam(learning_rate= 0.00001, decay= 1e-6)
model.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics = ['acc'])
checkpointer = ModelCheckpoint(filepath='./ResNet101_ART_epoch30_MassOnly.hdf5',
                            monitor='val_loss', verbose = 1,
                            save_best_only=True)
early_stopping = EarlyStopping(verbose=1, patience=15)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.00000001)

history_res101 = model.fit(ART_train_generator,
                    steps_per_epoch = 20,
                    epochs = 30,
                    verbose = 1,
                    validation_data = ART_val_generator,
                    callbacks = [checkpointer, early_stopping])

In [ ]:
!nvidia-smi

In [ ]:
!conda list

In [ ]:
result = model.evaluate(ART_test_generator)

In [ ]:
plt.plot(history_res101.history['acc'], label = 'train',)
plt.plot(history_res101.history['val_acc'], label = 'val')

plt.legend(loc = 'right')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

## VGG16

In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt 
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Layer, Flatten
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
img_size = 512
N_CLASSES = 2
base_model = VGG16(include_top = False,
                   weights = 'imagenet',
                   input_shape = (img_size,img_size,3))

last = base_model.layers[-2].output
x = GlobalAveragePooling2D()(last)
x = Dense(512,'relu')(x)
x = Dense(N_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=x)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])
model.summary()

In [ ]:
model_name = "vgg16_ART.h5"
checkpoint = ModelCheckpoint(model_name,
                            monitor="val_loss",
                            mode="min",
                            save_best_only = True,
                            verbose=1)

earlystopping = EarlyStopping(monitor='val_loss',min_delta = 0, patience = 5, verbose = 1, restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.00000001)

In [ ]:
history = model.fit(ART_train_generator,
                    epochs=100,
                    validation_data=ART_val_generator,
                    callbacks=[checkpoint,earlystopping,learning_rate_reduction])

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
result = model.evaluate(ART_test_generator)

## InceptionResNetV2

In [ ]:
from tensorflow.keras.applications import InceptionResNetV2

In [ ]:
img_size = 512
N_CLASSES = 2
base_model = InceptionResNetV2(include_top = False,
                   weights = 'imagenet',
                   input_shape = (img_size,img_size,3))

last = base_model.layers[-2].output
x = GlobalAveragePooling2D()(last)
x = Dense(512,'relu')(x)
x = Dense(N_CLASSES, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=x)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])
model.summary()

In [ ]:
model_name = "InceptionResNetV2_ART_epoch30.h5"
checkpoint = ModelCheckpoint(model_name,
                            monitor="val_loss",
                            mode="min",
                            save_best_only = True,
                            verbose=1)

earlystopping = EarlyStopping(monitor='val_loss',min_delta = 0, patience = 5, verbose = 1, restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.00000001)

In [ ]:
history = model.fit(ART_train_generator,
                    epochs=30,
                    validation_data=ART_val_generator,
                    callbacks=[checkpoint,earlystopping,learning_rate_reduction])

In [ ]:
model = tf.keras.models.load_model('InceptionResNetV2_ART_epoch30.h5')
model

In [ ]:
result = model.evaluate(ART_test_generator)

In [ ]:
result = model.evaluate(GCCT_test_generator)

## EfficientNetB0

In [ ]:
import tensorflow as tf
from tf.keras.applications.efficientnet import EfficientNetB0

### Cross validation         
change image data directory format to perform cross validation        
모델은 ResNet101 을 사용하자. 

In [ ]:
# create dataframe of file list
# 라벨별로 분리해둔 폴더에서 filename - label 쌍의 dataframe을 생성

ART_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_classified/TRAIN/ART/'
ART_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_classified/TEST/ART/'
# PRE_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_classified/TRAIN/PRE/'
# PRE_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_classified/TEST/PRE/'
# GCCT_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_classified/TRAIN/GCCT/'
# GCCT_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_classified/TEST/GCCT/'

# ART_train filename + label DF
ART_train_RCC = pd.DataFrame(os.listdir(ART_train_path + 'RCC/'))
ART_train_RCC['label'] = 'RCC'
ART_train_RCC.rename(columns = {0 : 'filename'}, inplace = True)

ART_train_AMLonco = pd.DataFrame(os.listdir(ART_train_path + 'AML + onco/'))
ART_train_AMLonco['label'] = 'AML + onco'
ART_train_AMLonco.rename(columns = {0 : 'filename'}, inplace = True)

ART_train_df = pd.concat([ART_train_RCC, ART_train_AMLonco], axis = 0)
ART_train_df.reset_index(drop = True, inplace = True)

# # PRE_train filename + label DF
# PRE_train_RCC = pd.DataFrame(os.listdir(PRE_train_path + 'RCC/'))
# PRE_train_RCC['label'] = 'RCC'
# PRE_train_RCC.rename(columns = {0 : 'filename'}, inplace = True)

# PRE_train_AMLonco = pd.DataFrame(os.listdir(PRE_train_path + 'AML + onco/'))
# PRE_train_AMLonco['label'] = 'AML + onco'
# PRE_train_AMLonco.rename(columns = {0 : 'filename'}, inplace = True)

# PRE_train_df = pd.concat([PRE_train_RCC, PRE_train_AMLonco], axis = 0)
# PRE_train_df.reset_index(drop = True, inplace = True)

# # GCCT_train filename + label DF
# GCCT_train_RCC = pd.DataFrame(os.listdir(GCCT_train_path + 'RCC/'))
# GCCT_train_RCC['label'] = 'RCC'
# GCCT_train_RCC.rename(columns = {0 : 'filename'}, inplace = True)

# GCCT_train_AMLonco = pd.DataFrame(os.listdir(GCCT_train_path + 'AML + onco/'))
# GCCT_train_AMLonco['label'] = 'AML + onco'
# GCCT_train_AMLonco.rename(columns = {0 : 'filename'}, inplace = True)

# GCCT_train_df = pd.concat([GCCT_train_RCC, GCCT_train_AMLonco], axis = 0)
# GCCT_train_df.reset_index(drop = True, inplace = True)

In [ ]:
ART_train_df

In [ ]:
# train, test 합쳐진 데이터 다시 나누기

ART_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_mass_total/ART/'
PRE_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_mass_total/PRE/'
GCCT_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_mass_total/GCCT/'

new_ART_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_traintest/TRAIN/ART/'
new_ART_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_traintest/TEST/ART/'
new_PRE_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_traintest/TRAIN/PRE/'
new_PRE_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_traintest/TEST/PRE/'
new_GCCT_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_traintest/TRAIN/GCCT/'
new_GCCT_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_traintest/TEST/GCCT/'

# make dir
def makedir(path): 
    if not os.path.exists(path): 
        os.mkdir(path)
        
dir = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_traintest/'
makedir(os.path.join(dir, "TRAIN"))
makedir(os.path.join(dir, "TEST"))

makedir(os.path.join(dir, "TRAIN", "ART"))
makedir(os.path.join(dir, "TEST", "ART"))

makedir(os.path.join(dir, "TRAIN", "PRE"))
makedir(os.path.join(dir, "TEST", "PRE"))

makedir(os.path.join(dir, "TRAIN", "GCCT"))
makedir(os.path.join(dir, "TEST", "GCCT"))


# create img name list
art_list = os.listdir(ART_path)
pre_list = os.listdir(PRE_path)
gcct_list = os.listdir(GCCT_path)


# ART
for img in tqdm(art_list): 
    case_id = img[0:10]
    if case_id[6:10] <= '0320': # train
        shutil.copy(ART_path + img, new_ART_train_path + img)
    else: # test
        shutil.copy(ART_path + img, new_ART_test_path + img)

# PRE
for img in tqdm(pre_list): 
    case_id = img[0:10]
    if case_id[6:10] <= '0320': # train
        shutil.copy(PRE_path + img, new_PRE_train_path + img)
    else: # test
        shutil.copy(PRE_path + img, new_PRE_test_path + img)
        
# GCCT
for img in tqdm(gcct_list): 
    case_id = img[0:10]
    if case_id[6:10] <= '0320': # train
        shutil.copy(GCCT_path + img, new_GCCT_train_path + img)
    else: # test
        shutil.copy(GCCT_path + img, new_GCCT_test_path + img)

In [ ]:
# image_shape = (512, 512, 3)
# res101_model = ResNet101(include_top=False, pooling='avg', weights='imagenet', input_shape = (image_shape))
# for layer in res101_model.layers:
#     layer.trainable = False

In [ ]:
image_shape = (512, 512, 3)
res101_model = ResNet101(include_top=False, pooling='avg', weights=None, input_shape = (image_shape))
for layer in res101_model.layers:
    layer.trainable = True

In [ ]:
# 모델 정의 - ResNet101
N_CLASSES = 2

model = Sequential()
model.add(res101_model)
model.add(Dropout(0.6))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(300))
model.add(BatchNormalization())
model.add(Dense(100))
model.add(BatchNormalization())
model.add(Dropout(0.6))
model.add(Dense(N_CLASSES, activation='softmax'))
model.summary()

In [ ]:
# gradcam 적용 위해서 VGG16으로 바꿔보기

base_model = VGG16(include_top = False,
                   weights = 'imagenet',
                   input_shape = (img_size,img_size,3))

last = base_model.layers[-2].output
x = GlobalAveragePooling2D()(last)
x = Dense(512,'relu')(x)
x = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=x)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy'])
model.summary()

In [ ]:
# # test dataset 은 다시 라벨별로 분리해둔 폴더를 사용

# ART_test_path = '/home/ncp/workspace/blocks3/zio_code/kidneyData_windowing/TEST/ART/'

# BATCH_SIZE = 32

# # create datagen
# create_datagen = ImageDataGenerator(rescale= 1./255.)


# # generate test data first
# ART_test_generator = create_datagen.flow_from_directory(ART_test_path,
#                                                    batch_size = BATCH_SIZE,
#                                                    target_size = (512, 512),
#                                                    class_mode = 'categorical')

In [ ]:
# ART # - TRAIN: 서울대, TEST: 부산+보라매일 경우
# 다시 라벨별 분리하지 않은 데이터 경로에서 작업 (덕선선생님이 만드신것)
# test dataset 은 다시 라벨별로 분리해둔 폴더를 사용
# 1007 learning rate reduction 추가
from tensorflow.keras.callbacks import ReduceLROnPlateau

BATCH_SIZE = 32

ART_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_traintest/TRAIN/ART/'
PRE_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_traintest/TRAIN/PRE/'
GCCT_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_traintest/TRAIN/GCCT/'

ART_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_classified/TEST/ART/'
PRE_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_classified/TEST/PRE/'
GCCT_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_classified/TEST/GCCT/'

# y is imbalance - KFold 대신 StratifiedKFold 사용
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

# create datagen
create_datagen = ImageDataGenerator(rescale= 1./255.)


# generate test data first
ART_test_generator = create_datagen.flow_from_directory(ART_test_path,
                                                   batch_size = BATCH_SIZE,
                                                   target_size = (512, 512),
                                                   class_mode = 'categorical')


## PERFORM TRAIN & VAL ## - ART

VALIDATION_ACCURACY = []
VALIDATION_LOSS = []
VALIDATION_AUC = []
val_acc = 0

fold_var = 1 # weight 나 모델 저장용

for train_index, val_index in skf.split(ART_train_df, ART_train_df['label']):  # Y의 분포 기준으로 나눔
    
    training_data = ART_train_df.iloc[train_index]
    validation_data = ART_train_df.iloc[val_index]
    
    train_data_generator = create_datagen.flow_from_dataframe(training_data, directory = ART_train_path, 
                                                             x_col = 'filename', y_col = 'label', 
                                                             class_mode = 'categorical', 
                                                             shuffle = True)
    valid_data_generator = create_datagen.flow_from_dataframe(validation_data, directory = ART_train_path, 
                                                             x_col = 'filename', y_col = 'label', 
                                                             class_mode = 'categorical', 
                                                             shuffle = True)
    
    optimizer = optimizers.Adam(learning_rate= 0.0001, decay= 1e-6)
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                           patience =3, 
                                           verbose =1,
                                            factor = 0.2,
                                            min_lr =0.0000001)
    model.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy', 'AUC'])
    checkpointer = ModelCheckpoint(filepath='./ResNet101_crossval_ART_epoch30.hdf5',
                            monitor='val_loss', verbose = 1,
                            save_best_only=True)
    early_stopping = EarlyStopping(verbose=1, patience=15)

    history_res101 = model.fit(train_data_generator,
                        epochs = 3,
                        verbose = 1,
                        validation_data = valid_data_generator,
                        callbacks = [checkpointer, early_stopping, learning_rate_reduction])
    
    # validation accuracy, loss
    results = model.evaluate(valid_data_generator)
    results = dict(zip(model.metrics_names, results))
    
    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])
    VALIDATION_AUC.append(results['auc'])
    
    tf.keras.backend.clear_session()
    
    fold_var += 1
    
    val_acc += results['accuracy'] / 10   # 평균 ACC
    
print("mean validation accuracy: ", val_acc)

In [ ]:
# concat json label dataframes

ART_label = pd.concat([ART_train_label, ART_val_label], axis = 0).reset_index(drop = True)
PRE_label = pd.concat([PRE_train_label, PRE_val_label], axis = 0).reset_index(drop = True)

In [ ]:
ART_label

In [ ]:
shutil.rmtree('/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_Shuffled_TrainTest/TRAIN/')
shutil.rmtree('/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_Shuffled_TrainTest/TEST/')

In [ ]:
# TRAIN+TEST 합쳐진 데이터에서 랜덤하게 20%만 분리해서 train_shuffle, test_shuffle로 다시 저장
# Train: ART, PRE, GCCT 만 분리하고, 안에 라벨은 생성 x (label df를 따로 사용)
# Test: 20% 분리 후 라벨까지 생성

ART_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly/ART/'
PRE_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly/PRE/'
# GCCT_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_massBoundingBox/GCCT/'

new_ART_train_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_Shuffled_TrainTest/TRAIN/ART/'
new_PRE_train_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_Shuffled_TrainTest/TRAIN/PRE/'
# new_GCCT_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_Shuffled_TrainTest/TRAIN/GCCT/'
new_ART_test_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_Shuffled_TrainTest/TEST/ART/'
new_PRE_test_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_Shuffled_TrainTest/TEST/PRE/'
# new_GCCT_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_Shuffled_TrainTest/TEST/GCCT/'

# make dir
def makedir(path): 
    if not os.path.exists(path): 
        os.mkdir(path)
        
dir = '/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_Shuffled_TrainTest/'
makedir(os.path.join(dir, "TRAIN"))
makedir(os.path.join(dir, "TEST"))

makedir(os.path.join(dir, "TRAIN", "ART"))
makedir(os.path.join(dir, "TRAIN", "PRE"))
makedir(os.path.join(dir, "TRAIN", "GCCT"))

makedir(os.path.join(dir, "TEST", "ART"))
makedir(os.path.join(dir, "TEST", "ART", "RCC"))
makedir(os.path.join(dir, "TEST", "ART", "AML + onco"))

makedir(os.path.join(dir, "TEST", "PRE"))
makedir(os.path.join(dir, "TEST", "PRE", "RCC"))
makedir(os.path.join(dir, "TEST", "PRE", "AML + onco"))

# makedir(os.path.join(dir, "TEST", "GCCT"))
# makedir(os.path.join(dir, "TEST", "GCCT", "RCC"))
# makedir(os.path.join(dir, "TEST", "GCCT", "AML + onco"))


# create img name list
art_list = os.listdir(ART_path)
pre_list = os.listdir(PRE_path)
# gcct_list = os.listdir(GCCT_path)


# split 20% and create train, test list
import random
# ART
art_rcc_list = os.listdir('/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_classified/ART/RCC/')
art_amlonco_list = os.listdir('/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_classified/ART/AML + onco/')

art_rcc_test_list = random.sample(art_rcc_list, int(round(len(art_rcc_list) * 0.2)))
art_amlonco_test_list = random.sample(art_amlonco_list, int(round(len(art_amlonco_list) * 0.2)))

art_test_list = list(set(art_rcc_test_list+art_amlonco_test_list))
art_train_list = list(set(art_rcc_list)-set(art_rcc_test_list)) + list(set(art_amlonco_list)-set(art_amlonco_test_list))

# PRE
pre_rcc_list = os.listdir('/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_classified/PRE/RCC/')
pre_amlonco_list = os.listdir('/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_classified/PRE/AML + onco/')

pre_rcc_test_list = random.sample(pre_rcc_list, int(round(len(pre_rcc_list) * 0.2)))
pre_amlonco_test_list = random.sample(pre_amlonco_list, int(round(len(pre_amlonco_list) * 0.2)))

pre_test_list = list(set(pre_rcc_test_list+pre_amlonco_test_list))
pre_train_list = list(set(pre_rcc_list)-set(pre_rcc_test_list)) + list(set(pre_amlonco_list)-set(pre_amlonco_test_list))

# # GCCT
# gcct_rcc_list = os.listdir('/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_classified_Train+Test/GCCT/RCC/')
# gcct_amlonco_list = os.listdir('/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_classified_Train+Test/GCCT/AML + onco/')

# gcct_rcc_test_list = random.sample(gcct_rcc_list, int(round(len(gcct_rcc_list) * 0.2)))
# gcct_amlonco_test_list = random.sample(gcct_amlonco_list, int(round(len(gcct_amlonco_list) * 0.2)))

# gcct_test_list = list(set(gcct_rcc_test_list+gcct_amlonco_test_list))
# gcct_train_list = list(set(gcct_rcc_list)-set(gcct_rcc_test_list)) + list(set(gcct_amlonco_list)-set(gcct_amlonco_test_list))


# ART train
for img in tqdm(art_list): 
    if img in art_train_list: 
        shutil.copy(ART_path + img, new_ART_train_path + img)

# PRE train
for img in tqdm(pre_list): 
    if img in pre_train_list: 
        shutil.copy(PRE_path + img, new_PRE_train_path + img)
        
# # GCCT train
# for img in tqdm(gcct_list): 
#     if img in gcct_train_list: 
#         shutil.copy(GCCT_path + img, new_GCCT_train_path + img)
        
# ART test
for img in tqdm(art_list): 
    if img in art_test_list: 
        case_id = img[0:10]
        label = ART_label.loc[ART_label['case_id'] == case_id, 'label'].unique().tolist()
        if label == ['RCC']: 
            shutil.copy(ART_path + img, new_ART_test_path + 'RCC/' + img)
        if label == ['AML + onco']: 
            shutil.copy(ART_path + img, new_ART_test_path + 'AML + onco/' + img)

# PRE test
for img in tqdm(pre_list): 
    if img in pre_test_list: 
        case_id = img[0:10]
        label = PRE_label.loc[PRE_label['case_id'] == case_id, 'label'].unique().tolist()
        if label == ['RCC']: 
            shutil.copy(PRE_path + img, new_PRE_test_path + 'RCC/' + img)
        if label == ['AML + onco']: 
            shutil.copy(PRE_path + img, new_PRE_test_path + 'AML + onco/' + img)
        
# # GCCT test
# for img in tqdm(gcct_list): 
#     if img in gcct_test_list: 
#         case_id = img[0:10]
#         label = PRE_label.loc[PRE_label['case_id'] == case_id, 'label'].unique().tolist()
#         if label == ['RCC']: 
#             shutil.copy(GCCT_path + img, new_GCCT_test_path + 'RCC/' + img)
#         if label == ['AML + onco']: 
#             shutil.copy(GCCT_path + img, new_GCCT_test_path + 'AML + onco/' + img)

In [ ]:
len(art_test_list)

In [ ]:
len(art_train_list)

In [ ]:
# create dataframe of file list
# 라벨별로 분리해둔 폴더에서 filename - label 쌍의 dataframe을 생성
# 1111 서울대+부산+보라매 합칠 시 실행 - 재 생성 필요함

ART_train_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_Shuffled_TrainTest/TRAIN/ART/'
PRE_train_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_Shuffled_TrainTest/TRAIN/PRE/'
# GCCT_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_Shuffled_TrainTest/TRAIN/GCCT/'

# ART_train filename + label DF
ART_train_df = pd.DataFrame(os.listdir(ART_train_path))
ART_train_df.rename(columns = {0 : 'filename'}, inplace = True)
ART_train_df['label'] = ''
for row in range(len(ART_train_df)): 
    case_id = ART_train_df['filename'][row][0:10]
    label = ART_label.loc[ART_label['case_id'] == case_id, 'label'].unique().tolist()
    if label == ['RCC']: 
        ART_train_df['label'][row] = 'RCC'
    if label == ['AML + onco']:
        ART_train_df['label'][row] = 'AML + onco'
        
ART_train_df.reset_index(drop = True, inplace = True)

# PRE_train filename + label DF
PRE_train_df = pd.DataFrame(os.listdir(PRE_train_path))
PRE_train_df.rename(columns = {0 : 'filename'}, inplace = True)
PRE_train_df['label'] = ''
for row in range(len(PRE_train_df)): 
    case_id = PRE_train_df['filename'][row][0:10]
    label = PRE_label.loc[PRE_label['case_id'] == case_id, 'label'].unique().tolist()
    if label == ['RCC']: 
        PRE_train_df['label'][row] = 'RCC'
    if label == ['AML + onco']:
        PRE_train_df['label'][row] = 'AML + onco'
        
PRE_train_df.reset_index(drop = True, inplace = True)

# # GCCT_train filename + label DF
# GCCT_train_df = pd.DataFrame(os.listdir(GCCT_train_path))
# GCCT_train_df.rename(columns = {0 : 'filename'}, inplace = True)
# GCCT_train_df['label'] = ''
# for row in range(len(GCCT_train_df)): 
#     case_id = GCCT_train_df['filename'][row][0:10]
#     label = PRE_label.loc[PRE_label['case_id'] == case_id, 'label'].unique().tolist()
#     if label == ['RCC']: 
#         GCCT_train_df['label'][row] = 'RCC'
#     if label == ['AML + onco']:
#         GCCT_train_df['label'][row] = 'AML + onco'
        
# GCCT_train_df.reset_index(drop = True, inplace = True)

In [ ]:
ART_train_df

In [ ]:
import sklearn

In [ ]:
# ART # - 서울대+부산+보라매 다 합쳐서 random하게 train test split한 데이터로 train
from tensorflow.keras.callbacks import ReduceLROnPlateau

BATCH_SIZE = 32

ART_train_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_Shuffled_TrainTest/TRAIN/ART/'
# PRE_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_Shuffled_TrainTest/TRAIN/PRE/'
# GCCT_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_Shuffled_TrainTest/TRAIN/GCCT/'

ART_test_path = '/home/ncp/workspace/blocks1/kidneyData_windowing_massOnly_Shuffled_TrainTest/TEST/ART/'
# PRE_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_Shuffled_TrainTest/TEST/PRE/'
# GCCT_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_Shuffled_TrainTest/TEST/GCCT/'

# y is imbalance - KFold 대신 StratifiedKFold 사용
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

# create datagen
create_datagen = ImageDataGenerator(rescale= 1./255.)


# generate test data first
ART_test_generator = create_datagen.flow_from_directory(ART_test_path,
                                                   batch_size = BATCH_SIZE,
                                                   target_size = (512, 512),
                                                   class_mode = 'categorical')


## PERFORM TRAIN & VAL ## - ART

VALIDATION_ACCURACY = []
VALIDATION_LOSS = []
VALIDATION_AUC = []
val_acc = 0

fold_var = 1 # weight 나 모델 저장용

for train_index, val_index in skf.split(ART_train_df, ART_train_df['label']):  # Y의 분포 기준으로 나눔
    
    training_data = ART_train_df.iloc[train_index]
    validation_data = ART_train_df.iloc[val_index]
    
    train_data_generator = create_datagen.flow_from_dataframe(training_data, directory = ART_train_path, 
                                                             x_col = 'filename', y_col = 'label', 
                                                             class_mode = 'categorical', 
                                                             shuffle = True)
    valid_data_generator = create_datagen.flow_from_dataframe(validation_data, directory = ART_train_path, 
                                                             x_col = 'filename', y_col = 'label', 
                                                             class_mode = 'categorical', 
                                                             shuffle = True)
    
    optimizer = optimizers.Adam(learning_rate= 0.0001, decay= 1e-6)
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                           patience =3, 
                                           verbose =1,
                                            factor = 0.2,
                                            min_lr =0.0000001)
    model.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy', 'AUC'])
    checkpointer = ModelCheckpoint(filepath='./RES101_crossval_ART_epoch30_afterShuffle_MassOnly.hdf5',
                            monitor='val_loss', verbose = 1,
                            save_best_only=True)
    early_stopping = EarlyStopping(verbose=1, patience=15)

    history_res101 = model.fit(train_data_generator,
                        epochs = 30,
                        verbose = 1,
                        validation_data = valid_data_generator,
                        callbacks = [checkpointer, early_stopping, learning_rate_reduction])
    
    # validation accuracy, loss
    results = model.evaluate(valid_data_generator)
    results = dict(zip(model.metrics_names, results))
    
    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])
    VALIDATION_AUC.append(results['auc'])
    
    tf.keras.backend.clear_session()
    
    fold_var += 1
    
    val_acc += results['accuracy'] / 10   # 평균 ACC
    
print("mean validation accuracy: ", val_acc)

In [ ]:
# path_resnet_ART = '/home/ncp/workspace/blocks3/zio_code/cancer_configuration_ART-ResNet101_crossval.hdf5'
# ART_resnet = tf.keras.models.load_model(path_resnet_ART)

In [ ]:
result = model.evaluate(ART_test_generator)

In [ ]:
# generate test data first
GCCT_test_generator = create_datagen.flow_from_directory(GCCT_test_path,
                                                   batch_size = BATCH_SIZE,
                                                   target_size = (512, 512),
                                                   class_mode = 'categorical')

In [ ]:
result_gcct = model.evaluate(GCCT_test_generator)

In [ ]:
plt.plot(VALIDATION_ACCURACY, label = 'val_accuracy')
plt.plot(VALIDATION_LOSS, label = 'val_loss')

plt.legend(loc = 'right')
plt.xlabel('folds')
plt.ylabel('accuracy')
plt.title('Val acc and loss by folds')
plt.show()

In [ ]:
# auc

plt.plot(VALIDATION_AUC, label = 'val_auc', color = 'g')

plt.legend(loc = 'right')
plt.xlabel('folds')
plt.ylabel('AUC')
plt.title('Val AUC by folds')
plt.show()

In [ ]:
## grad cam 적용 ##

import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt 
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Layer, Flatten
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report

In [ ]:
y_val = ART_test_generator.classes
y_pred = model.predict(ART_test_generator)
y_pred = np.argmax(y_pred,axis=1)

In [ ]:
print(classification_report(y_val,y_pred))

In [ ]:
class_indices = ART_test_generator.class_indices
indices = {v:k for k,v in class_indices.items()}

In [ ]:
filenames = ART_test_generator.filenames

In [ ]:
val_df = pd.DataFrame()
val_df['filename'] = filenames
val_df['actual'] = y_val
val_df['predicted'] = y_pred
val_df['actual'] = val_df['actual'].apply(lambda x: indices[x])
val_df['predicted'] = val_df['predicted'].apply(lambda x: indices[x])
val_df.loc[val_df['actual']==val_df['predicted'],'Same'] = True
val_df.loc[val_df['actual']!=val_df['predicted'],'Same'] = False
val_df.head(10)

In [ ]:
val_df = val_df.sample(frac=1).reset_index(drop=True)

In [ ]:
len(val_df)

In [ ]:
img_size = 512
def readImage(path):
    img = load_img(path,color_mode='rgb',target_size=(img_size,img_size))
    img = img_to_array(img)
    img = img/255.
    
    return img

def display_images(temp_df, image_type): # image_type: one of ART, PRE, GCCT
    temp_df = temp_df.reset_index(drop=True)
    plt.figure(figsize = (20 , 20))
    n = 0
    for i in range(15):
        n+=1
        plt.subplot(5 , 5, n)
        plt.subplots_adjust(hspace = 0.5 , wspace = 0.3)
        image = readImage(f"/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_Shuffled_TrainTest/TEST/{image_type}/{temp_df.filename[i]}")
        plt.imshow(image)
        plt.title(f'A: {temp_df.actual[i]} P: {temp_df.predicted[i]}')

In [ ]:
# correctly classified

display_images(val_df[val_df['Same']==True], 'ART')

In [ ]:
# misclassified
display_images(val_df[val_df['Same']!=True], 'ART')

In [ ]:
def make_gradcam_heatmap(img_array, model, pred_index=None):
    
    grad_model = Model(inputs=model.inputs, outputs=[model.layers[-4].output, model.output])

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy(), preds

In [ ]:
import matplotlib.cm as cm

In [ ]:
def gradCAMImage(image, image_type):
    path = "/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_Shuffled_TrainTest/TEST/{}/{}".format(image_type, image)
    img = readImage(path)
    img = np.expand_dims(img,axis=0)
    heatmap,preds = make_gradcam_heatmap(img,model)

    img = load_img(path)
    img = img_to_array(img)
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = tf.keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = tf.keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * 0.8 + img
    superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)
    
    return superimposed_img

In [ ]:
def gradcam_of_images(correct_class, image_type):
    grad_images = []
    title = []
    temp_df = val_df[val_df['Same']==correct_class]
    temp_df = temp_df.reset_index(drop=True)
    for i in range(15):
        image = temp_df.filename[i]
        grad_image = gradCAMImage(image, image_type)
        grad_images.append(grad_image)
        title.append(f"A: {temp_df.actual[i]} P: {temp_df.predicted[i]}")

    return grad_images, title

In [ ]:
correctly_classified, c_titles = gradcam_of_images(correct_class=True, 'ART')
misclassified, m_titles = gradcam_of_images(correct_class=False, 'ART')

In [ ]:
def display_heatmaps(classified_images,titles):
    plt.figure(figsize = (20 , 20))
    n = 0
    for i in range(15):
        n+=1
        plt.subplot(5 , 5, n)
        plt.subplots_adjust(hspace = 0.5 , wspace = 0.3)
        plt.imshow(classified_images[i])
        plt.title(titles[i])
    plt.show()

### GRAD-CAM: Correctly classified

In [ ]:
display_heatmaps(correctly_classified,c_titles)

### GRAD-CAM: Incorrectly classified

In [ ]:
display_heatmaps(misclassified,m_titles)

In [ ]:
# # roc curve - check if works

# from sklearn import datasets
# iris = datasets.load_iris()
# X = iris.data
# y = iris.target
# X, y = X[y != 2], y[y != 2]
# n_samples, n_features = X.shape

# # Add noisy features
# random_state = np.random.RandomState(0)
# X = np.c_[X, random_state.randn(n_samples, 200*n_features)]

In [ ]:
# import matplotlib.pyplot as plt
# from sklearn.model_selection import StratifiedKFold

# from sklearn import svm
# from sklearn.metrics import auc
# from sklearn.metrics import RocCurveDisplay

# # Run classifier with cross-validation and plot ROC curves
# cv = StratifiedKFold(n_splits = 6)
# classifier = svm.SVC(kernel = "linear", probability = True, random_state = random_state)

# tprs = []
# aucs = []
# mean_fpr = np.linspace(0, 1, 100)

# fig, ax = plt.subplots()
# for i, (train, test) in enumerate(cv.split(X, y)): 
#     viz = RocCurveDisplay.from_estimator(
#     model, 
#     train_data_generator, 
#     valid_data_generator, 
#     name = "ROC fold {}".format(fold_var), 
#     alpha = 0.3, 
#     lw = 1, 
#     ax = ax
# )

In [ ]:
# # roc curve

# acc = history_res101.history['accuracy']
# val_acc = history_res101.history['val_accuracy']

# AUC = history_res101.history['AUC']
# val_AUC = history_res101.history['val_auc']

# epochs = range(1, len(acc) + 1)

# plt.plot(epochs, AUC, 'g', label = 'AUC')
# plt.plot(epochs, val_AUC, 'b', label = 'Val_AUC')
# plt.xlabel('No.of epochs')
# plt.ylabel('AUC')
# plt.title('Training and validation AUC')
# plt.legend()
# plt.figure()

In [ ]:
# GCCT
# 다시 라벨별 분리하지 않은 데이터 경로에서 작업 (덕선선생님이 만드신것)
# test dataset 은 다시 라벨별로 분리해둔 폴더를 사용
# 1007 learning rate reduction 추가
from tensorflow.keras.callbacks import ReduceLROnPlateau

BATCH_SIZE = 32

ART_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_traintest/TRAIN/ART/'
PRE_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_traintest/TRAIN/PRE/'
GCCT_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_traintest/TRAIN/GCCT/'

ART_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_classified/TEST/ART/'
PRE_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_classified/TEST/PRE/'
GCCT_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_classified/TEST/GCCT/'

# y is imbalance - KFold 대신 StratifiedKFold 사용
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

# create datagen
create_datagen = ImageDataGenerator(rescale= 1./255.)


# generate test data first
GCCT_test_generator = create_datagen.flow_from_directory(GCCT_test_path,
                                                   batch_size = BATCH_SIZE,
                                                   target_size = (512, 512),
                                                   class_mode = 'categorical')


## PERFORM TRAIN & VAL ## - ART

VALIDATION_ACCURACY = []
VALIDATION_LOSS = []
VALIDATION_AUC = []
val_acc = 0

fold_var = 1 # weight 나 모델 저장용

for train_index, val_index in skf.split(GCCT_train_df, GCCT_train_df['label']):  # Y의 분포 기준으로 나눔
    
    training_data = GCCT_train_df.iloc[train_index]
    validation_data = GCCT_train_df.iloc[val_index]
    
    train_data_generator = create_datagen.flow_from_dataframe(training_data, directory = GCCT_train_path, 
                                                             x_col = 'filename', y_col = 'label', 
                                                             class_mode = 'categorical', 
                                                             shuffle = True)
    valid_data_generator = create_datagen.flow_from_dataframe(validation_data, directory = GCCT_train_path, 
                                                             x_col = 'filename', y_col = 'label', 
                                                             class_mode = 'categorical', 
                                                             shuffle = True)
    
    optimizer = optimizers.Adam(learning_rate= 0.0001, decay= 1e-6)
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                           patience =3, 
                                           verbose =1,
                                            factor = 0.2,
                                            min_lr =0.0000001)
    model.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy', 'AUC'])
    checkpointer = ModelCheckpoint(filepath='./ResNet101_crossval_GCCT_epoch30.hdf5',
                            monitor='val_loss', verbose = 1,
                            save_best_only=True)
    early_stopping = EarlyStopping(verbose=1, patience=15)

    history_res101 = model.fit(train_data_generator,
                        epochs = 3,
                        verbose = 1,
                        validation_data = valid_data_generator,
                        callbacks = [checkpointer, early_stopping, learning_rate_reduction])
    
    # validation accuracy, loss
    results = model.evaluate(valid_data_generator)
    results = dict(zip(model.metrics_names, results))
    
    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])
    VALIDATION_AUC.append(results['auc'])
    
    tf.keras.backend.clear_session()
    
    fold_var += 1
    
    val_acc += results['accuracy'] / 10   # 평균 ACC
    
print("mean validation accuracy: ", val_acc)

In [ ]:
# GCCT # - 서울대, 보라매, 부산 다 합쳐서 random하게 split 한 데이터로 train
from tensorflow.keras.callbacks import ReduceLROnPlateau

BATCH_SIZE = 32

ART_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_total_Shuffled_TrainTest/TRAIN/ART/'
PRE_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_total_Shuffled_TrainTest/TRAIN/PRE/'
GCCT_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_total_Shuffled_TrainTest/TRAIN/GCCT/'

ART_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_total_Shuffled_TrainTest/TEST/ART/'
PRE_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_total_Shuffled_TrainTest/TEST/PRE/'
GCCT_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_mass_total_Shuffled_TrainTest/TEST/GCCT/'

# y is imbalance - KFold 대신 StratifiedKFold 사용
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

# create datagen
create_datagen = ImageDataGenerator(rescale= 1./255.)


# generate test data first
GCCT_test_generator = create_datagen.flow_from_directory(GCCT_test_path,
                                                   batch_size = BATCH_SIZE,
                                                   target_size = (512, 512),
                                                   class_mode = 'categorical')


## PERFORM TRAIN & VAL ## - ART

VALIDATION_ACCURACY = []
VALIDATION_LOSS = []
VALIDATION_AUC = []
val_acc = 0

fold_var = 1 # weight 나 모델 저장용

for train_index, val_index in skf.split(GCCT_train_df, GCCT_train_df['label']):  # Y의 분포 기준으로 나눔
    
    training_data = GCCT_train_df.iloc[train_index]
    validation_data = GCCT_train_df.iloc[val_index]
    
    train_data_generator = create_datagen.flow_from_dataframe(training_data, directory = GCCT_train_path, 
                                                             x_col = 'filename', y_col = 'label', 
                                                             class_mode = 'categorical', 
                                                             shuffle = True)
    valid_data_generator = create_datagen.flow_from_dataframe(validation_data, directory = GCCT_train_path, 
                                                             x_col = 'filename', y_col = 'label', 
                                                             class_mode = 'categorical', 
                                                             shuffle = True)
    
    optimizer = optimizers.Adam(learning_rate= 0.0001, decay= 1e-6)
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                           patience =3, 
                                           verbose =1,
                                            factor = 0.2,
                                            min_lr =0.0000001)
    model.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy', 'AUC'])
    checkpointer = ModelCheckpoint(filepath='./ResNet101_crossval_GCCT_epoch30_afterShuffle.hdf5',
                            monitor='val_loss', verbose = 1,
                            save_best_only=True)
    early_stopping = EarlyStopping(verbose=1, patience=15)

    history_res101 = model.fit(train_data_generator,
                        epochs = 30,
                        verbose = 1,
                        validation_data = valid_data_generator,
                        callbacks = [checkpointer, early_stopping, learning_rate_reduction])
    
    # validation accuracy, loss
    results = model.evaluate(valid_data_generator)
    results = dict(zip(model.metrics_names, results))
    
    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])
    VALIDATION_AUC.append(results['auc'])
    
    tf.keras.backend.clear_session()
    
    fold_var += 1
    
    val_acc += results['accuracy'] / 10   # 평균 ACC
    
print("mean validation accuracy: ", val_acc)

In [ ]:
result = model.evaluate(GCCT_test_generator)

In [ ]:
plt.plot(VALIDATION_ACCURACY, label = 'val_accuracy')
plt.plot(VALIDATION_LOSS, label = 'val_loss')

plt.legend(loc = 'right')
plt.xlabel('folds')
plt.ylabel('accuracy')
plt.title('Val acc and loss by folds')
plt.show()

In [ ]:
# auc

plt.plot(VALIDATION_AUC, label = 'val_auc', color = 'g')

plt.legend(loc = 'right')
plt.xlabel('folds')
plt.ylabel('AUC')
plt.title('Val AUC by folds')
plt.show()

In [ ]:
# PRE # - 서울대+부산+보라매 다 합쳐서 random하게 train test split한 데이터로 train
from tensorflow.keras.callbacks import ReduceLROnPlateau

BATCH_SIZE = 32

ART_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_Shuffled_TrainTest/TRAIN/ART/'
PRE_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_Shuffled_TrainTest/TRAIN/PRE/'
# GCCT_train_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_Shuffled_TrainTest/TRAIN/GCCT/'

ART_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_Shuffled_TrainTest/TEST/ART/'
PRE_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_Shuffled_TrainTest/TEST/PRE/'
# GCCT_test_path = '/home/ncp/workspace/blocks3/kidneyData_windowing_massBoundingBox_Shuffled_TrainTest/TEST/GCCT/'

# y is imbalance - KFold 대신 StratifiedKFold 사용
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

# create datagen
create_datagen = ImageDataGenerator(rescale= 1./255.)


# generate test data first
PRE_test_generator = create_datagen.flow_from_directory(PRE_test_path,
                                                   batch_size = BATCH_SIZE,
                                                   target_size = (512, 512),
                                                   class_mode = 'categorical')


## PERFORM TRAIN & VAL ## - ART

VALIDATION_ACCURACY = []
VALIDATION_LOSS = []
VALIDATION_AUC = []
val_acc = 0

fold_var = 1 # weight 나 모델 저장용

for train_index, val_index in skf.split(PRE_train_df, PRE_train_df['label']):  # Y의 분포 기준으로 나눔
    
    training_data = PRE_train_df.iloc[train_index]
    validation_data = PRE_train_df.iloc[val_index]
    
    train_data_generator = create_datagen.flow_from_dataframe(training_data, directory = PRE_train_path, 
                                                             x_col = 'filename', y_col = 'label', 
                                                             class_mode = 'categorical', 
                                                             shuffle = True)
    valid_data_generator = create_datagen.flow_from_dataframe(validation_data, directory = PRE_train_path, 
                                                             x_col = 'filename', y_col = 'label', 
                                                             class_mode = 'categorical', 
                                                             shuffle = True)
    
    optimizer = optimizers.Adam(learning_rate= 0.0001, decay= 1e-6)
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss',
                                           patience =3, 
                                           verbose =1,
                                            factor = 0.2,
                                            min_lr =0.0000001)
    model.compile(optimizer=optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy', 'AUC'])
    checkpointer = ModelCheckpoint(filepath='./VGG16_crossval_PRE_epoch30_afterShuffle_BoundingBoxAdded.hdf5',
                            monitor='val_loss', verbose = 1,
                            save_best_only=True)
    early_stopping = EarlyStopping(verbose=1, patience=15)

    history_res101 = model.fit(train_data_generator,
                        epochs = 30,
                        verbose = 1,
                        validation_data = valid_data_generator,
                        callbacks = [checkpointer, early_stopping, learning_rate_reduction])
    
    # validation accuracy, loss
    results = model.evaluate(valid_data_generator)
    results = dict(zip(model.metrics_names, results))
    
    VALIDATION_ACCURACY.append(results['accuracy'])
    VALIDATION_LOSS.append(results['loss'])
    VALIDATION_AUC.append(results['auc'])
    
    tf.keras.backend.clear_session()
    
    fold_var += 1
    
    val_acc += results['accuracy'] / 10   # 평균 ACC
    
print("mean validation accuracy: ", val_acc)

In [ ]:
result = model.evaluate(PRE_test_generator)

In [ ]:
plt.plot(VALIDATION_ACCURACY, label = 'val_accuracy',)
plt.plot(VALIDATION_LOSS, label = 'val_loss')

plt.legend(loc = 'right')
plt.xlabel('folds')
plt.ylabel('accuracy')
plt.title('Val acc and loss by folds')
plt.show()

In [ ]:
# auc

plt.plot(VALIDATION_AUC, label = 'val_auc', color = 'g')

plt.legend(loc = 'right')
plt.xlabel('folds')
plt.ylabel('AUC')
plt.title('Val AUC by folds')
plt.show()

In [ ]:
## grad cam 적용 ##

import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt 
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Layer, Flatten
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report
import matplotlib.cm as cm

In [ ]:
y_val = PRE_test_generator.classes
y_pred = model.predict(PRE_test_generator)
y_pred = np.argmax(y_pred,axis=1)

In [ ]:
print(classification_report(y_val,y_pred))

In [ ]:
class_indices = PRE_test_generator.class_indices
indices = {v:k for k,v in class_indices.items()}

In [ ]:
filenames = PRE_test_generator.filenames

In [ ]:
val_df = pd.DataFrame()
val_df['filename'] = filenames
val_df['actual'] = y_val
val_df['predicted'] = y_pred
val_df['actual'] = val_df['actual'].apply(lambda x: indices[x])
val_df['predicted'] = val_df['predicted'].apply(lambda x: indices[x])
val_df.loc[val_df['actual']==val_df['predicted'],'Same'] = True
val_df.loc[val_df['actual']!=val_df['predicted'],'Same'] = False
val_df.head(10)

In [ ]:
val_df = val_df.sample(frac=1).reset_index(drop=True)

In [ ]:
len(val_df)

In [ ]:
# correctly classified

display_images(val_df[val_df['Same']==True], 'PRE')

In [ ]:
# misclassified
display_images(val_df[val_df['Same']!=True], 'PRE')

In [ ]:
correctly_classified, c_titles = gradcam_of_images(correct_class=True, image_type = 'PRE')
misclassified, m_titles = gradcam_of_images(correct_class=False, image_type = 'PRE')

### GRAD-CAM: Correctly classified

In [ ]:
display_heatmaps(correctly_classified,c_titles)

### GRAD-CAM: Incorrectly classified

In [ ]:
display_heatmaps(misclassified,m_titles)

In [ ]:
path_resnet_PRE = '/home/ncp/workspace/blocks3/zio_code/cancer_configuration_PRE-ResNet101_crossval.hdf5'
PRE_resnet = tf.keras.models.load_model(path_resnet_PRE)

In [ ]:
PRE_resnet()

In [ ]:
result_PRE = model.evaluate(PRE_test_generator)

In [ ]:
plt.plot(history_res101_PRE.history['accuracy'], label = 'train',)
plt.plot(history_res101_PRE.history['val_accuracy'], label = 'val')

plt.legend(loc = 'right')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()